# Validation with Pydantic

https://www.youtube.com/watch?v=SW1ZdqH0rRQ&t=872s <br>
https://www.youtube.com/watch?v=6ppxd9lp-X0

## Importing LLM model and Libraries

In [3]:
import os
import transformers

from getpass import getpass
from secret_key import huggingface_key
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_key

In [ ]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


In [ ]:
"""https://huggingface.co/deepset/roberta-base-squad2"""
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
#importing module for langchain and kor

from typing import List, Optional

from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI



from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

import pandas as pd
from pydantic import BaseModel, Field, validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain


In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.llms import HuggingFaceHub

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline


In [ ]:
#importing llm model

#from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id = model,
    huggingfacehub_api_token = huggingface_key
)

## Loading  and Splitting Document

In [ ]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#insert filename you want to extract information from 
filename = "data/authorize_doc/StarlinkGen2_FCC-22-91A1.txt"

In [ ]:
#loading the document
def import_document(filename):
    encodings = ['utf-8', 'ISO-8859-1', 'utf-16', 'ascii', 'cp1252']
    for enc in encodings:
        try:
            with open(filename, 'r', encoding=enc) as file:
                document_text = file.read()
            return document_text
        except UnicodeDecodeError:
            continue
        except FileNotFoundError:
            print(f"Error: File '{filename}' not found.")
            return None
        except Exception as e:
            print(f"Error occurred while importing the document: {e}")
            return None
    print(f"Error: Could not decode file with any of the tried encodings: {encodings}")
    return None

document = import_document(filename)
if document is not None:
    print("Document content:")
    print(document)

Let's briefly explore the document

In [ ]:
len(document)

In [ ]:
#this is token count in textacy
import textacy
doc = textacy.make_spacy_doc(document, lang="en_core_web_sm")
print(doc._.preview)

In [ ]:
from textacy import text_stats as ts

# Number of words and number of unique words
print("Number of words: ", ts.n_words(doc))
print("Number of unique words: ", ts.n_unique_words(doc))

# Entropy of words in the document- measures how much informations produced on the average of the word
print("Entropy: ", ts.entropy(doc))

# Compute the Type-Token Ratio (TTR) of doc_or_token,a direct ratio of the number of unique words (types) of all words (token)
print("Diversity: ", ts.diversity.ttr(doc))

# Flesch Kincaid grade level: readability tests designed to indicate how difficult a passage is
print("Flesch Kincaid: ",ts.flesch_kincaid_grade_level(doc))

Let's now split the document into chunks 

In [ ]:
#split the document into chunks
doc = Document(page_content = document)
split_docs = RecursiveCharacterTextSplitter().split_documents([doc])

## Building the Pydantic Model

It Kor doumentation it says that the Validation doe NO imply that extraction was correct.
Validation only implies that the data was returned in the correct shape and meets all validation criteria.
This does not mean that the LLM didn't make up some information

we can use pydantic to sip the invalid data

In [ ]:
from pydantic import BaseModel, Field, validator, ValidationError
from typing import Optional, List, Union
import re

### Pydantic Class

In [ ]:
class OrbitEnv(BaseModel):
    const_name: str = Field(
        description="The satellite constellation name for which the company applied to deploy or operate",
    )
    orbit_type: str = Field(
        description="The orbit type into which the satellites will be launched"
    )
    application: str = Field(
        description="The application or services that the satellites would provide"
    )
    date_50: str = Field(
        description="The date when the company is order to deploy and operate half of its satellites"
    )
    date_100: str = Field(
        description="The date when the company is order to deploy and operate all of its remaining satellites."
    )
    total_sat_const: int = Field(
        description="The concluding total number of satellites that the company has been authorized to deploy and operate for the constellation"
    )
    altitude: Optional[List[float]]= Field(
        description="The granted altitudes of the satellites that the company has been authorized to deploy"
    )
    #can add to description: that the inclination would be in degree - maybe delete the words altitude
    #The granted inclination of the satellites that the company has been authorized to deploy, in degree
    inclination: Optional[List[float]] = Field(
        description="The granted inclination of the satellites that the company has been authorized to deploy, respective to the altitudes"
    )
    number_orb_plane: Optional[List[int]] = Field(
        description="The number of orbital planes, respective to the altitudes and inclination, that the company has been authorized to deploy"
    )
    total_sat_per_orb_plane: Optional[List[int]]= Field(
        description="The specific count of satellites located in each individual orbital plane. This count refers to the total number of satellites within one orbital plane, and it can vary from plane to plane based on the altitude and inclination, and if not mentioned in text, 'total_sat_per_alt_incl' divide by 'number_orb_plane' will give this value"
    )
    total_sat_per_alt_incl: Optional[List[int]] = Field(
        description="The total number of satellites at a specific altitude and inclination across all orbital planes sharing these characteristics. This count represents the overall number of satellites with the specified altitude and inclination parameters, and if not mentioned in the text, the multiplication of 'number_orb_plane' and 'total_sat_per_orb_plane' will give this value"
    )
    orbit_shape: Optional[str] = Field(
        description="The shape of the orbital plane whether its circular, elliptical or are not mention in the document"
    )
    operational_lifetime : Optional[str] = Field(
        description="The operational lifetime of the satellite in the constellation in years"
    )


    @validator("const_name", "orbit_type", "application")
    def validate_name(cls, v):
        if not re.match("^[a-zA-Z\s().,-]*$", v):
            raise ValueError("The field can only contain alphabetic characters, spaces, parentheses, periods, commas and hyphen.")
        return v
    
    @validator("total_sat_const", "number_orb_plane", "total_sat_per_orb_plane", "total_sat_per_alt_incl", "operational_lifetime")
    def validate_whole_number(cls, v):
        if isinstance(v, list):
            if not all(isinstance(i, int) for i in v):
                raise ValueError("All elements of the list must be whole numbers.")
        elif v is not None and not isinstance(v, int):
            raise ValueError("The field must be a whole number.")
        return v

    @validator("altitude", "inclination")
    def validate_number(cls, v):
        if isinstance(v, list):
            if not all(isinstance(i, (int, float)) for i in v):
                raise ValueError("All elements of the list must be numbers (integer or decimal).")
        elif v is not None and not isinstance(v, (int, float)):
            raise ValueError("The field must be a number (integer or decimal).")
        return v

    @validator("orbit_shape")
    def validate_orbit_shape(cls, v):
        if not re.match("^[a-zA-Z\s]*$", v):
            raise ValueError("orbit_shape can only contain alphabetic characters and spaces.")
        return v
    


'Optional[str]' means that the field can be either a str(string) or None effectively making it optional

'str' means that it is mandatory

### Schema and Examples

In [ ]:
schema, extraction_validator = from_pydantic(
    OrbitEnv,
    description="Extract the Orbital Environment information of a Satellite Constellation from the authorized document. Include details such as the company name, orbit type, application, dates for 50 percent and 100 percent satellite launches, total number of authorized satellites, altitude, inclination, number of orbital planes, number of satellites per plane, and orbit shape",
    examples=[
        (
            """In this Order and Authorization, we grant, to the extent set forth below, the request of Kuiper Systems LLC (Kuiper or Amazon) to deploy a non-geostationary satellite orbit (NGSO) system to provide service using certain Fixed-Satellite Service (FSS).
                Operating 3,372 satellites in 102 orbital planes at altitudes of 590 km, 610 km, and 630 km in a circular orbit.
                At 590 km, 30 orbital planes with 28 satellites per plane for a total of 840 satellites at inclination of 33 degree.
                At 610 km, 42 orbital planes with 36 satellites per plane for a total of 1512 satellites at inclination of 42 degree.
                At 630 km, 30 orbital planes with 34 satellites per plane for a total of 1020 satellite at inclination of 51.9 degree.
                The constellation are require to launch and operate 50 percent of its satellites no later than July 30, 2026, and Kuiper must launch the remaining space stations necessary to complete its authorized service constellation, place them in their assigned orbits, and operate each of them in accordance with the authorization no later than July 30, 2029.""",
                
            {"const_name": "Kuiper System LLC", "orbit_type": "non-geostationary satellite orbit (NGSO)", "application": "Fixed-Satellite Service (FSS)", "date_50": "July 30, 2026", "date_100": "July 30, 2029", "total_sat_const": 3372, "altitude": [590, 610, 630],  "inclination": [33, 42, 51.9], "number_orb_plane": [30, 42, 30], "total_sat_per_orb_plane": [28, 36, 34], "total_sat_per_alt_incl": [840, 1512, 1020], "orbit_shape": "circular"}
        ),
        (
            "Iridium must launch 50 percent of the maximum number of proposed space stations, place them in the assigned orbits, and operate them in accordance with this grant no later than November 12,2028, and must launch the remaining space stations necessary to complete its authorized service constellation, place them in their assigned orbits, and operate them in accordance with the authorization no later than May 16,2030.",
            {"const_name": "Iridium","date_50":"November 12,2028,","date_100":"May 16,2030"}
        ),
        (
            "They must launch 50 percent of the maximum number of proposed space stations, place them in the assigned orbits, and operate them in accordance with this grant of U.S. market access no later than December 31,1989, and must launch the remaining space stations necessary to complete its authorized service constellation, place them in their assigned orbits, and operate them in accordance with the grant of U.S. market access no later than December 21,1997.",
            {"date_50":"December 31,1989","date_100":"November 21,1997"}
        ),
        (
            "In this Order and Declaratory Ruling, we grant in part and defer in part the petition for declaratory ruling of WorldVu Satellites Limited (OneWeb) for modification of its grant of U.S. market access for a its satellite constellation authorized by the United Kingdom. As modified, the constellation will operate with four fewer satellites, reduced from 720 to 716 satellites.",
            {"const_name": "WorldVu Satellites Limited (OneWeb)", "total_sat_const": 716}
        ),
        (
            "They sought Commission approval for a non-geostationary satellite orbit (NGSO) system to provide fixed-satellite service (FSS) in the United States.",
            {"orbit_type": "non-geostationay satellite orbit (NGSO)", "application": "fixed-satellite service (FSS)"}
        ),
        (
            """The proposed Telesat system is set to feature a robust constellation of 124 satellites.
            A set of six orbital planes, each inclined at 99.5 degrees, will host nine satellites per plane at an approximate altitude of 1,000 kilometers.
            Additionally, seven more orbital planes, each tilted at 37.4 degrees, will carry another group of satellites, with each plane accommodating ten satellites at a higher altitude of approximately 1,248 kilometers.
            It's noteworthy that all satellites will occupy a circular orbit, ensuring systematic and efficient coverage.""",
            {"const_name": "Telesat", "total_sat_const": 124, "altitude": [1000, 1248], "inclination": [99.5, 37.4], "number_orb_plane": [6, 7], "total_sat_per_orb_plane": [9, 10], "total_sat_per_alt_incl": [54, 70], "orbit_shape": "circular"}
        ),
        #different between total_sat_per_orb_plane and total_sat_per_alt_incl
        (
            "20 orbital planes with 28 satellites per plane for a total of 560 satellites at inclination of 33 degree will be placed at an altitude approximately 800 km.",
            {"altitude": 800, "inclination": 33, "number_orb_plane": 20, "total_sat_per_orb_plane": 28, "total_sat_per_alt_incl": 560}
        ),
        #total_sat_per_alt_incl = number_orb_plane x total_sat_per_orb_plane
        (
            "8 orbital plane containing 15 satellites each which are inclined at 56 degree with altitude of 700 kilometers",
            {"altitude": 700, "inclination": 56, "number_orb_plane": 8, "total_sat_per_orb_plane": 15, "total_sat_per_alt_incl": 120}
        ),
        #total_sat_per_orb_plane = total_sat_per_alt_incl x number_orb_plane
        (
            "72 of the satellites will be distributed equally and place at 6 orbital planes, which are inclined 99.5 degrees, satellites will be at an approximate altitude of 1,000 kilometers",
            {"altitude": 1000, "inclination": 99.5, "number_orb_plane": 6, "total_sat_per_orb_plane": 12, "total_sat_per_alt_incl": 72}
        ),
        #operational_lifetime
        (
            "The operational lifetime for the satellite in the constellation in 10 years",
            {"operational_lifetime": 10}
        ),

    ],
    many=True,
)

will provide more examples

""" #maneuverable and spin-stabilized
(
    """Each satellite in the constellation is equipped with propulsion, enabling it to perform maneuvers to avoid collisions and navigate to its designated operational orbit.
    Additionally, the satellites also have spin stabilizers, ensuring their stability during orbital operation.""",
    {"maneuverable": "y", "spin_stabilized": "y"}
), """

'many' parameter determine whether the funciton should expect to work with a single instance of the object or multiple instances

- If many=False (the default), the schema expects to validate a single object of the class defined in the function call (OrbitEnv in your case).
- If many=True, the schema expects to validate a list of objects of the class defined in the function call.

### Creating Chain

In [ ]:
chain = create_extraction_chain(
    llm,
    schema,
    encoder_or_encoder_class="json",
    validator=extraction_validator,
    input_formatter="triple_quotes",
)

#csv does support list , but json is not as accurate as csv

looking at what are the prompt and istruction pass to the LLM model

In [ ]:
print(chain.prompt.format_prompt(text="[user input]").to_string())

seeing how much it cost us

In [ ]:
with get_openai_callback() as cb:
    document_extraction_results = await extract_from_documents(
        chain, split_docs, max_concurrency=5, use_uid=False, return_exceptions=True
    )
    #split_docs is where we input the document we want to extract
    #use_uid: parameter that determine whether or not to use a unique identifier (uid)when processesing document
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")

seeing the raw return

In [ ]:
document_extraction_results

### Creating a dataframe

In [ ]:
import pandas as pd

def generate_dataframe(json_data):
    # Prepare an empty list to store all OrbitEnv data
    data = []

    for record in json_data:
        # Check if the record is a dictionary. If not, print an error and continue to the next record
        if not isinstance(record, dict):
            print(f"Error encountered: {record}")
            continue
        
        orbitenv_list = record.get('data', {}).get('orbitenv', [])
        for orbitenv in orbitenv_list:
            data.append([
                orbitenv.get('const_name', ''),
                orbitenv.get('orbit_type', ''),
                orbitenv.get('application', ''),
                orbitenv.get('date_50', ''),
                orbitenv.get('date_100', ''),
                orbitenv.get('total_sat_const', ''),
                orbitenv.get('altitude', '') or '',
                orbitenv.get('inclination', '') or '',
                orbitenv.get('number_orb_plane', '') or '',
                orbitenv.get('total_sat_per_orb_plane', '') or '',
                orbitenv.get('total_sat_per_alt_incl', '') or '',
                orbitenv.get('orbit_shape', ''),
                orbitenv.get('operational_lifetime', '')
            ])

    # Convert the list into a DataFrame
    df = pd.DataFrame(data, columns=['constellationName', 'orbitType', 'application','date50', 'date100', 'totalSatelliteNumber', 'altitudes','inclination', 'numberOrbPlane', 'totalSatellitePerOrbPlane','totalSatellitePerAltIncl', 'orbShape', 'operationalLifetime'])

    # Replace various values with None
    df.replace(['','-',0,'Null', 'null', 'Not Mentioned', 'Not mentioned', 'not mentioned', 'unknown', 'Unknown','N/A'], None, inplace=True)
    
    return df

# Usage:
df = generate_dataframe(document_extraction_results)


In [ ]:
df

In [ ]:
df.shape

## Most Frequent in Each Column

In [ ]:
import pandas as pd
import json
import numpy as np
import re

def find_most_frequent(df: pd.DataFrame) -> dict:
    most_frequent_dict = {}
    for column in df.columns:
        column_without_none = df[column].dropna()
        if not column_without_none.empty:
            mode = column_without_none.mode()
            if len(mode) > 1:
                most_frequent_dict[column] = {"message": "Multiple modes found", "modes": mode.tolist()}
            else:
                most_frequent_dict[column] = mode[0]
        else:
            most_frequent_dict[column] = None
    return most_frequent_dict

def convert(o):
    if isinstance(o, np.generic):
        return o.item()
    raise TypeError

def convert_to_json(data: dict) -> str:
    try:
        json_data = json.dumps(data, default=convert)
        return json_data
    except TypeError:
        return json.dumps({"error": "Failed to serialize data"})

result = find_most_frequent(df)
result
#returninng dictionary key-value pair, mutable , can be add, remove, change element

In [ ]:
# do and if statement here to find date_500 and date_100

In [ ]:
print(type(result))

### Converting to Json and exporting

In [ ]:
json_data = convert_to_json(result)

name = result.get('constellationName', {}).get('modes', [None])[0] if isinstance(result.get('constellationName', {}), dict) else result.get('constellationName', None)


In [ ]:
name

In [ ]:
print(type(json_data))

In [ ]:
json_data

In [ ]:
if name is not None:
    name = re.sub(r'\W+', '_', name)
    filename = f'output/{name}_{model}_data.json'

    with open(filename, 'w+') as txt_file:
        txt_file.write(json_data)


### yet to do
- for the total number of satellite in constellation you can do if there are multiple mode found match the value with the sum of all array in total number of satellite (per altitude/inclination) - if it match that is your total number of satellite in constellation
- adding in maneuverable and spin stabilisation as a field and operational lifetime - manueverable and stabilisation is really hard to get it right - also add orbit epoch
- using different LLM model
- using different company order authorize document -done
- using different type of document - schedule S or techical document
- how to measure validation (intrisic and extrinsic) - validate if the extracted info is in and at what paragraph - or hallucination
- make this a model? - putting input text - output json fresh  - calculation coding - true process all column 29

- maybe before doing extraction - do a sentiment analysis for the whole document - to see if the purpose constallation are fully granted, partially granted ot denied 

- need to extract also the release date
